In [2]:
import os
import csv

import pandas as pd


In [3]:
# load
book_rating_df = pd.read_csv("BX-Book-Ratings.csv", encoding="cp1251", sep=";")
book_rating_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
# Drop implicit ratings
book_rating_df = book_rating_df[book_rating_df["Book-Rating"] != 0]
book_rating_df.head()

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6


In [18]:
books_df = pd.read_csv("BX-Books.csv", encoding="cp1251", sep=";", escapechar="\\")
books_df = books_df[["ISBN", "Book-Title", "Book-Author"]]
books_df.head(3)

,ISBN,Book-Title,Book-Author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este


In [19]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ISBN         271379 non-null  object
 1   Book-Title   271379 non-null  object
 2   Book-Author  271378 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB


In [21]:
books_df = books_df.astype({'ISBN': 'string', 'Book-Title': 'string', 'Book-Author': 'string'})
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ISBN         271379 non-null  string
 1   Book-Title   271379 non-null  string
 2   Book-Author  271378 non-null  string
dtypes: string(3)
memory usage: 6.2 MB


In [28]:
books_df.fillna("Unknown", inplace=True)

# First we will convert both the book titles and book authors
# into lowercase (to improve the effects of later deduplication).
books_df["Book-Title"] = books_df["Book-Title"].apply(str.lower)
books_df["Book-Author"] = books_df["Book-Author"].apply(str.lower)

# TODO - resolve abbreviations (j.r.r. Tolkien OR j. r. r. Tolkien)

In [29]:
books_df.groupby(["Book-Title", "Book-Author"]).agg("count").reset_index().sort_values("ISBN", ascending=False).head(10)

,Book-Title,Book-Author,ISBN
112631,little women,louisa may alcott,23
246064,wuthering heights,emily bronte,22
9680,adventures of huckleberry finn,mark twain,20
147906,pride and prejudice,jane austen,19
213949,the secret garden,frances hodgson burnett,17
56628,dracula,bram stoker,16
79747,great expectations,charles dickens,16
98574,jane eyre,charlotte bronte,16
25046,black beauty,anna sewell,14
61919,emma,jane austen,14


We can see that we have a lot of book duplicates (different releases and editions of the same book title by the same author).
We will attempt to resolve that once we merge the reviews with the book data.

In [36]:
df = book_rating_df.merge(books_df, on="ISBN")
df = df.astype({'ISBN': 'string', 'Book-Title': 'string', 'Book-Author': 'string'})
df.head(3)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author
0,276726,0155061224,5,rites of passage,judith rae
1,276729,052165615X,3,help!: level 1,philip prowse
2,276729,0521795028,6,the amsterdam connection : level 4 (cambridge ...,sue leather


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 383852 entries, 0 to 383851
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   User-ID      383852 non-null  int64 
 1   ISBN         383852 non-null  string
 2   Book-Rating  383852 non-null  int64 
 3   Book-Title   383852 non-null  string
 4   Book-Author  383852 non-null  string
dtypes: int64(2), string(3)
memory usage: 17.6 MB


In [ ]:
# In the case of much bigger dataset (this one has about 18MB in memory),
# we would like to avoid recomputing the whole pivot table.
# To do so, we could incrementally load (in batches) new review data

# For incremental load of additional review data, I would need to store additional fields,
# in order to resolve the deduplicated ISBNs. E.g. list of the same different ISBNs for the same BookTitle+Author.
# Also, for the recomputation of the average ratings I would also need to store amount of reviews used to calculate the average

# For sake of simplicity, In this task, I will only consider the initial load and preprocessing of the available dataset...

In [70]:
dfc = df.groupby(["Book-Title", "Book-Author", "ISBN"]).count().reset_index()
# Take the ISBNs with the highest amount of reviews
dfcs = dfc.sort_values(["Book-Title", "Book-Author", "User-ID"], ascending=False).groupby(["Book-Title", "Book-Author"]).head(1)
# And just a small test:
dfcs[dfcs["Book-Title"]=='the fellowship of the ring (the lord of the rings, part 1)']
# as we can see, there are still some issues with abbreviations...

,Book-Title,Book-Author,ISBN,User-ID,Book-Rating
116093,the fellowship of the ring (the lord of the ri...,j.r.r. tolkien,0345339703,131,131
116087,the fellowship of the ring (the lord of the ri...,j. r. r. tolkien,0618002227,43,43


In [33]:
book_rating_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 433671 entries, 1 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   User-ID      433671 non-null  int64 
 1   ISBN         433671 non-null  object
 2   Book-Rating  433671 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 13.2+ MB
